## (30%) Finding similiarity
Distance based metrics are commonly used methods to create similiarity between two items. This similarity will be applied in content-based recommendation. They also have been used for a long time in Information Retrieval methods.

Here is a good article on [Distance Metrics for Fun and Profit](http://www.benfrederickson.com/distance-metrics/) by Ben on better understanding of these metrics

Could you based on the provided dataset (stories.csv) to generate similarity matrix between each story? The i, j, cell of this matrix will store the similarity value between story i and story j. You can define your own similarity measures.   

#### Solutions

In [1]:
import pandas as pd

In [2]:
stories = pd.read_csv("C:/Users/shihyuch/rouseguy_recsys_workshop/recommendation-master/data/stories.csv", low_memory=False)

In [3]:
stories.head()

,story,title,score,user
0,14274033,"Uber is valued at $70B, you can get it at $999",503,appoets
1,14107522,Jeff Bezos? Annual Letter,547,djyaz1200
2,13647190,India has banned disposable plastic in Delhi,671,SimplyUseless
3,13309610,Startup Puts Everything You Need for a Two-Acr...,511,kungfudoi
4,13860890,The Uber Bombshell About to Drop,1089,dantiberian


## Creating More features from Title

In [ ]:
! pip install spacy

In [26]:
! python -m spacy download en

In [7]:
# Lets use spact
import spacy
nlp = spacy.load('en')

In [9]:
title0 = nlp(stories.title[0])
title1 = nlp(stories.title[1])

In [10]:
title0, title1 

(Uber is valued at $70B, you can get it at $999, Jeff Bezos? Annual Letter)

In [11]:
len(title0.vector), len(title1.vector)

(384, 384)

In [12]:
title0.similarity(title1)

0.2867952452610529

In [13]:
story_similarity = []

The below code is time consuming - and so, we will run it only for the first 100 titles

In [14]:
%%time 

for story_row in stories.title[:100]:
    for story_column in stories.title[:100]:
        story_sim = nlp(story_row).similarity(nlp(story_column))
        story_similarity.append([story_row, story_column, story_sim])

Wall time: 4min 58s


In [15]:
story_similarity = pd.DataFrame(story_similarity,
                               columns = ["story1", "story2", "similarity"] )

In [16]:
story_similarity.head()

,story1,story2,similarity
0,"Uber is valued at $70B, you can get it at $999","Uber is valued at $70B, you can get it at $999",1.000000
1,"Uber is valued at $70B, you can get it at $999",Jeff Bezos? Annual Letter,0.286795
2,"Uber is valued at $70B, you can get it at $999",India has banned disposable plastic in Delhi,0.479615
3,"Uber is valued at $70B, you can get it at $999",Startup Puts Everything You Need for a Two-Acr...,0.460328
4,"Uber is valued at $70B, you can get it at $999",The Uber Bombshell About to Drop,0.413896


In [17]:
similarity_matrix = pd.pivot(story_similarity.story1,
        story_similarity.story2,
        story_similarity.similarity)

In [18]:
similarity_matrix.head()

story2,1Password Travel Mode: Protect your data when crossing borders,94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery,?Startup? asks internship applicant to build their app before phone screen,?The moon blew up without warning and for no apparent reason?,A crashed advertisement reveals logs of a facial recognition system,A lawsuit over Costco golf balls,A rising sentiment that IBM?s Watson can?t deliver on its promises,AMA: NY AG Schneiderman on net neutrality and protecting our voice in government,Accidentally destroyed production database on first day of a job,Afraid of Makefiles? Don't be,...,"Welcome, ACLU",What Is Ethereum?,What the CIA WikiLeaks Dump Tells Us: Encryption Works,Whistleblower uncovers London police hacking of journalists and protestors,Why F.E.A.R.?s AI is still the best in first-person shooters,Why Should I Start a Startup?,Why Slack is inappropriate for open source communications,Why We Must Fight for the Right to Repair Our Electronics,Why do many math books have so much detail and so little enlightenment? (2010),YC will hold interviews in Vancouver for founders who can?t get US visas
story1,,,,,,,,,,,,,,,,,,,,,
1Password Travel Mode: Protect your data when crossing borders,1.000000,0.615669,0.526267,0.282099,0.452962,0.603753,0.568304,0.663224,0.422880,0.401323,...,0.349826,0.288586,0.647402,0.606815,0.481515,0.525660,0.537459,0.509209,0.423268,0.533180
94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery,0.615669,1.000000,0.231294,0.283840,0.307012,0.545492,0.386210,0.640681,0.345800,0.270838,...,0.361119,0.217021,0.624808,0.441978,0.374290,0.340811,0.388952,0.438548,0.188818,0.367617
?Startup? asks internship applicant to build their app before phone screen,0.526267,0.231294,1.000000,0.663362,0.595468,0.513366,0.664473,0.599533,0.723266,0.446101,...,0.291643,0.350131,0.444095,0.375469,0.546044,0.483524,0.588525,0.493852,0.657745,0.499230
?The moon blew up without warning and for no apparent reason?,0.282099,0.283840,0.663362,1.000000,0.620172,0.493994,0.460750,0.599025,0.719966,0.208914,...,0.258612,0.325860,0.346406,0.286706,0.475740,0.373692,0.436561,0.419300,0.528632,0.269890
A crashed advertisement reveals logs of a facial recognition system,0.452962,0.307012,0.595468,0.620172,1.000000,0.639534,0.715270,0.599737,0.757002,0.091097,...,0.094465,0.087147,0.378254,0.574337,0.609272,0.134910,0.584835,0.187485,0.450068,0.394573


In [25]:
test = similarity_matrix.nlargest(3, '94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery')

In [27]:
test.shape

(3, 100)

In [29]:
test['94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery']

story1
94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery       1.000000
DeepMind and Blizzard Open StarCraft II as an AI Research Environment         0.883313
Mastering Chess and Shogi by Self-Play with General Reinforcement Learning    0.845477
Name: 94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery, dtype: float64

## (40%) Model - Matrix Completion by Alternating Least Square (ALS)

Since in practical, customers may refuse to rate some items, study following materials. 

For a better and visual understanding of Matrix Factorization Techniques, read the following links
- The original paper on [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)
- [Finding Similar Music using Matrix Factorization](http://www.benfrederickson.com/matrix-factorization/) by Ben Frederickson
- [Intro to Implicit Matrix Factorization: Classic ALS with Sketchfab Models](http://blog.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)

We will apply ALS method to fill those missing values (put as 0) for following matrix, a rating matrix for six customers (rows number, say 1,2,3,4,5,6) to seven items (columns number, say A,B,C,D,E,F,G):

R = np.array([[5, 0, 5, 0, 0, 1, 2], [0, 4, 0, 0, 2, 2, 1], [1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 1, 1, 0], [0, 2, 1, 0, 0, 1, 0], [0, 0, 3, 0, 1, 2, 0]])

**Decomposition of the rating matrix**  
R -> m users, n items  
Row in P -> user's affinity to the features (m users, f latent factors/features)  
Row in Q -> item's relation to the features (n items, f latent factors/features)  

- P indicates how much user likes f latent factors  
- Q indicates how much one item obtains f latent factors
- The dot product indicates how much user likes item  
- The decomposition automatically ranks features by their impact on the ratings
- Features may not be intuitive though !
- Model has hyperparameters (regularization, learning rate)

Your task is to write a function using iterative method, e.g., stochastic gradient descent, to factorize
matrix as P with dimension (6 x K) and Q with dimension (K x 7), where K is the number of latent features. 
After you get two factorized matrix P and Q, try to multiply them back, could you recommend the top three items of second customers based on this new rating matrix rating scores (0 value cells will have rating scores now). 


#### Solutions

In [2]:
import numpy as np
np.set_printoptions(precision=2)

In [3]:
"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""

def als_matrix_factorization(R, K=2, steps=5000, alpha=0.0002, beta=0.02):
    N = len(R)
    M = len(R[0])
    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)
    Q = Q.T
    
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

In [4]:
R = np.array([[5, 0, 5, 0, 0, 1, 2], 
              [0, 4, 0, 0, 2, 2, 1], 
              [1, 0, 0, 1, 0, 1, 0], 
              [0, 0, 1, 0, 1, 1, 0],
              [0, 2, 1, 0, 0, 1, 0],
              [0, 0, 3, 0, 1, 2, 0]])

In [9]:
# Run ALS
nP, nQ = als_matrix_factorization(R, K=2)

In [10]:
nP.dot(nQ.T)

array([[4.95, 5.19, 4.85, 2.88, 2.89, 1.29, 1.98],
       [3.23, 4.09, 3.63, 1.88, 1.85, 1.92, 1.04],
       [1.26, 1.55, 1.39, 0.73, 0.72, 0.68, 0.42],
       [1.13, 1.47, 1.3 , 0.66, 0.65, 0.73, 0.35],
       [1.29, 1.66, 1.47, 0.75, 0.74, 0.81, 0.41],
       [2.29, 3.3 , 2.84, 1.33, 1.29, 1.97, 0.6 ]])

In [7]:
nQ

array([[0.73, 1.8 ],
       [2.04, 0.74],
       [0.99, 1.71],
       [0.8 , 0.51],
       [0.95, 0.09],
       [1.18, 0.05],
       [0.36, 0.69]])

In [8]:
nP

array([[0.79, 2.45],
       [1.78, 0.53],
       [0.86, 0.24],
       [0.82, 0.16],
       [0.92, 0.07],
       [1.4 , 0.91]])

### For the second customer, the top 3 items order based on score from high to low is 
### B, C, A

## (20%) Learning to Rank

**Bayesian Personalized Ranking pairwise loss:**

Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.

More details can be found for BPR at 

https://www.coursera.org/lecture/matrix-factorization/personalized-ranking-with-daniel-kluver-s3XJo

https://www.slideshare.net/zenogantner/bayesian-personalized-ranking-for-nonuniformly-sampled-items

**Weighted Approximate-Rank Pairwise loss**:  
Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found. Useful when only positive interactions are present and optimising the top of the recommendation list (precision@k) is desired.WARP deals with (user, positive item, negative item) triplets. 

See following for WARP

https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a


This procedure yields roughly the following algorithm:

- For a given (user, positive item pair), sample a negative item at random from all the remaining items. Compute predictions for both items; if the negative item’s prediction exceeds that of the positive item plus a margin, perform a gradient update to rank the positive item higher and the negative item lower. If there is no rank violation, continue sampling negative items until a violation is found.

- If you found a violating negative example at the first try, make a large gradient update: this indicates that a lot of negative items are ranked higher than positives items given the current state of the model, and the model must be updated by a large amount. If it took a lot of sampling to find a violating example, perform a small update: the model is likely close to the optimum and should be updated at a low rate.

You can install lightfm package and use its fetch_movielens data sets.

In [2]:
#Get the data

import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

**Movie lens dataset:**

![](img/movielens.png)
GroupLens Research has collected and made available rating data sets from the MovieLens web site (http://movielens.org). The data sets were collected over various periods of time, depending on the size of the set.

http://grouplens.org/datasets/movielens/

**Fetch movielens 100k dataset**

The dataset contains 100,000 interactions from 1000 users on 1700 movies, and is exhaustively described
#in its README http://files.grouplens.org/datasets/movielens/ml-100k-README.txt

This data set consists of:
- 100,000 ratings (1-5) from 943 users on 1682 movies. 
- Each user has rated at least 20 movies. 
- Simple demographic info for the users (age, gender, occupation, zip)

### Questions 1: 

Using lightfm package builtin evaluation methods, e.g., precision_at_k and auc_score
to evaluate recommendation performance from Bayesian Personalized Ranking pairwise loss, and Weighted Approximate-Rank Pairwise loss.  

### Questions 2: 

Can you build a recommendation system based on Weighted Approximate-Rank Pairwise loss and perform recommendation for used with id [3, 25, 450]? You should show both known preference of used and predictions results (recommendatdions).
        

#### Solutions

In [4]:
movielens = fetch_movielens() 


In [5]:
#print movie lens 
for key, value in movielens.items():
    print(key, value)

item_labels ['Toy Story (1995)' 'GoldenEye (1995)' 'Four Rooms (1995)' ...,
 'Sliding Doors (1998)' 'You So Crazy (1994)'
 'Scream of Stone (Schrei aus Stein) (1991)']
test   (0, 19)	4
  (0, 32)	4
  (0, 60)	4
  (0, 116)	3
  (0, 154)	2
  (0, 159)	4
  (0, 170)	5
  (0, 188)	3
  (0, 201)	5
  (0, 264)	4
  (1, 12)	4
  (1, 49)	5
  (1, 250)	5
  (1, 279)	3
  (1, 280)	3
  (1, 289)	3
  (1, 291)	4
  (1, 296)	4
  (1, 311)	3
  (1, 313)	1
  (2, 244)	1
  (2, 293)	2
  (2, 322)	2
  (2, 327)	5
  (2, 330)	4
  :	:
  (940, 180)	5
  (940, 256)	4
  (940, 257)	4
  (940, 474)	4
  (940, 992)	4
  (941, 116)	4
  (941, 199)	4
  (941, 260)	4
  (941, 322)	3
  (941, 422)	5
  (941, 426)	5
  (941, 486)	4
  (941, 583)	4
  (941, 603)	4
  (941, 614)	3
  (942, 10)	4
  (942, 57)	4
  (942, 110)	4
  (942, 185)	5
  (942, 214)	5
  (942, 231)	4
  (942, 355)	4
  (942, 569)	1
  (942, 807)	4
  (942, 1066)	2
train   (0, 0)	5
  (0, 1)	3
  (0, 2)	4
  (0, 3)	3
  (0, 4)	3
  (0, 5)	5
  (0, 6)	4
  (0, 7)	1
  (0, 8)	5
  (0, 9)	3
  (0, 10)	2

In [6]:
train = movielens['train']
test = movielens['test']

#### BPR Model

We’ll use two metrics of accuracy: precision@k and ROC AUC

Both are ranking metrics: to compute them, we’ll be constructing recommendation lists for all of our users, and checking the ranking of known positive movies. 

For precision at k we’ll be looking at whether they are within the first k results on the list; for AUC, we’ll be calculating the probability that any known positive is higher on the list than a random negative example.

In [8]:
model = LightFM(learning_rate=0.05, loss='bpr')

In [9]:
model.fit(train, epochs=10)

In [10]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

In [11]:
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

In [12]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))


Precision: train 0.42, test 0.06.
AUC: train 0.84, test 0.82.


#### WARP Model

In [13]:
model = LightFM(learning_rate=0.05, loss='warp')

In [14]:
model.fit_partial(train, epochs=10)

In [15]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

In [16]:
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

In [17]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.60, test 0.11.
AUC: train 0.93, test 0.90.


### Prototype Rec Sys

In [18]:
#Let's quickly prototype a movie recommendation system

In [19]:
import numpy as np
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

In [20]:
data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 19048 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 2153 stored elements in COOrdinate format>


In [21]:
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)

CPU times: user 627 ms, sys: 7.69 ms, total: 635 ms
Wall time: 751 ms


In [22]:
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

Train precision: 0.40
Test precision: 0.05


Let's sample a couple of users and get their recommendations. 

To make predictions for given user, we pass the id of that user and the ids of all products we want predictions for into the predict method.

In [23]:
def sample_recommendation(model, data, user_ids):
    
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)
        

In [24]:
sample_recommendation(model, data, [3, 25, 450]) 

User 3
     Known positives:
        Contact (1997)
        Air Force One (1997)
        In & Out (1997)
     Recommended:
        Kiss the Girls (1997)
        G.I. Jane (1997)
        Lost Highway (1997)
User 25
     Known positives:
        Fargo (1996)
        Godfather, The (1972)
        L.A. Confidential (1997)
     Recommended:
        Fargo (1996)
        L.A. Confidential (1997)
        Star Wars (1977)
User 450
     Known positives:
        Event Horizon (1997)
        Scream (1996)
        Conspiracy Theory (1997)
     Recommended:
        Scream (1996)
        Game, The (1997)
        Air Force One (1997)


## (10%) Can you implement Discounted cumulative gain (DCG) and Normalized DCG. Then use example of ranking list to demonstrate your implementation.

#### Solutions

In [43]:
import math
import numpy as np


def find_dcg(element_list):
    """
    Discounted Cumulative Gain (DCG)
    The definition of DCG can be found in this paper:
        Azzah Al-Maskari, Mark Sanderson, and Paul Clough. 2007.
        "The relationship between IR effectiveness measures and user satisfaction."
    Parameters:
        element_list - a list of ranks Ex: [5,4,2,2,1]
    Returns:
        score
    """
    score = 0.0
    for order, rank in enumerate(element_list):
        score += float(rank)/np.log2((order+2))
    return score


def find_ndcg(reference, hypothesis):
    """
    Normalized Discounted Cumulative Gain (nDCG)
    Normalized version of DCG:
        nDCG = DCG(hypothesis)/DCG(reference)
    Parameters:
        reference   - a gold standard (perfect) ordering Ex: [5,4,3,2,1]
        hypothesis  - a proposed ordering Ex: [5,2,2,3,1]
    Returns:
        ndcg_score  - normalized score
    """

    return find_dcg(hypothesis)/find_dcg(reference)


In [44]:
print (find_dcg([3,2,3,0,1,2]))
print (find_dcg([3,3,3,2,2,2]))

6.861126688593502
8.740262365546284


In [45]:
reference = [3,3,3,2,2,2] # Ideal case
hypothesis = [3,2,3,0,1,2]
print(find_ndcg(reference, hypothesis))

0.785002371969948
